<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment_Feedforward_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import shutil
import tensorflow as tf
import string
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
print(tf.__version__)


2.6.0


# Part 1: IMDB sentiment analysis

## Loading the dataset

In [2]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
data = tf.keras.utils.get_file ('aclImdb_v1',
                                url,
                                untar = True,
                                cache_dir = '.',
                                cache_subdir = '')
data_dir = os.path.join (os.path.dirname(data), 'aclImdb')
print(os.listdir(data_dir))

['imdb.vocab', 'test', 'README', 'imdbEr.txt', 'train']


In [3]:
train_dir = os.path.join (data_dir, 'train')
print(os.listdir(train_dir))

['neg', 'urls_neg.txt', 'urls_pos.txt', 'unsup', 'pos', 'urls_unsup.txt', 'unsupBow.feat', 'labeledBow.feat']


In [4]:
# Loading data from the directory
batch_size = 32
seed = 42
raw_train = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                        batch_size =batch_size,
                                                        validation_split = 0.2,
                                                        subset = 'training',
                                                        seed = seed)
raw_val = tf.keras.utils.text_dataset_from_directory ('aclImdb/train',
                                                      batch_size = batch_size,
                                                      validation_split = 0.2,
                                                      subset = 'validation',
                                                      seed = seed)
raw_test = tf.keras.utils.text_dataset_from_directory ('aclImdb/test',
                                                       batch_size = batch_size)

Found 75000 files belonging to 3 classes.
Using 60000 files for training.
Found 75000 files belonging to 3 classes.
Using 15000 files for validation.
Found 25000 files belonging to 2 classes.


# Text representation

In [5]:
def custom_preprocessing (text):
  lowercase = tf.strings.lower (text)
  stripped_html = tf.strings.regex_replace (lowercase,'<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), 
                                  '')
  
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(standardize = custom_preprocessing,
                                           max_tokens = max_features,
                                           output_mode = 'int',
                                           output_sequence_length = sequence_length)
# Extracting features for vectorizing using training set
train_text = raw_train.map (lambda x, y: x)
vectorize_layer.adapt(train_text)

# Defining a function for fitting vectorizer function/layer to vectorize text (review)
def fitting_vectorizer (text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer (text), label

# storing text batch and label batch
text_batch, label_batch = next(iter(raw_train))

## print an instance with vectorized review and label for observing
print ('REVIEW:', text_batch[0])
print('LABEL:', raw_train.class_names[label_batch[0]] )


REVIEW: tf.Tensor(b"I bought this thinking I was gonna see Olivier Gruner kick some butt for 90 minutes but no the reasonably impressive cast including Michael Madson, Michael Ironside and Martin Kove are all wasted in mere cameos as they make way for the introduction of new action star Dan Anderson. The film also known in another name as Last line of defence 2 an apparant sequal to another film Gruner was in as the lead as a different character. What is the point? the first which I have not seen is apparantly about an alien or something, this is completley different featuring a story about a disgraced army guy (can't remeber who he worked for) who needs to rasie 5 million for a special new cancer treatment that will save his son and other children. Michael Ironside plays a wealthy business man who is of course a dirty scoundral who is swimming in dirty ill-gotten money. He laughs at Dan Andersons character and refuses to give it. Dan decides to steal it from him with the help of some 

In [6]:
# print an example of vectorized data
print ('Vocabulary size: ', len(vectorize_layer.get_vocabulary()))
for i in range (90, 100):
  print ('{} ------> {}'.format(i, vectorize_layer.get_vocabulary()[i]))

Vocabulary size:  10000
90 ------> them
91 ------> films
92 ------> movies
93 ------> then
94 ------> make
95 ------> made
96 ------> way
97 ------> could
98 ------> too
99 ------> after


In [7]:
train = raw_train.map(fitting_vectorizer)
val = raw_val.map(fitting_vectorizer)
test = raw_test.map(fitting_vectorizer)


In [8]:
# Configure the dataset for performance
autotune = tf.data.AUTOTUNE
train = train.cache().prefetch (buffer_size = autotune)
val = val.cache().prefetch (buffer_size = autotune)
test = test.cache().prefetch (buffer_size = autotune)

# Building a neural network classifier

In [12]:
# Creating the model
embedding_dim = 16 
model = tf.keras.Sequential([layers.Embedding(max_features + 1, embedding_dim),
                             layers.Dropout(0.2),
                             layers.GlobalAveragePooling1D(),
                             layers.Dropout(0.2),
                             layers.Dense(1)])
print(model.summary())

# configure the model uisng optimizer and loss function
model.compile(loss = losses.BinaryCrossentropy(from_logits = True),
              optimizer = 'adam',
              metrics = tf.metrics.BinaryAccuracy(threshold = 0.0 )) ## Why threshold = 0.0??

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160016    
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# training the model
epochs = 10
history = model.fit(train,
                    validation_data = val,
                    epochs = epochs)

Epoch 1/10
1875/1875 [==============================] - 15s 7ms/step - loss: -15.4567 - binary_accuracy: 0.1663 - val_loss: -43.8020 - val_binary_accuracy: 0.1681
Epoch 2/10
1875/1875 [==============================] - 14s 7ms/step - loss: -97.1702 - binary_accuracy: 0.1663 - val_loss: -153.9084 - val_binary_accuracy: 0.1681
Epoch 3/10
1875/1875 [==============================] - 14s 7ms/step - loss: -235.8746 - binary_accuracy: 0.1663 - val_loss: -312.2938 - val_binary_accuracy: 0.1681
Epoch 4/10
1875/1875 [==============================] - 14s 8ms/step - loss: -421.1499 - binary_accuracy: 0.1663 - val_loss: -514.2200 - val_binary_accuracy: 0.1681
Epoch 5/10
1875/1875 [==============================] - 14s 8ms/step - loss: -650.7604 - binary_accuracy: 0.1663 - val_loss: -758.3814 - val_binary_accuracy: 0.1681
Epoch 6/10
1875/1875 [==============================] - 14s 8ms/step - loss: -922.1535 - binary_accuracy: 0.1663 - val_loss: -1043.9900 - val_binary_accuracy: 0.1681
Epoch 7/10
1

In [11]:
# testing the model
loss, accuracy = model.evaluate(test)
print('Testing performance:\n Loss: {} - Accuracy: {}'. format(loss, accuracy))

782/782 [==============================] - 8s 10ms/step - loss: 2658.4441 - binary_accuracy: 0.5000
Testing performance:
 Loss: 2658.444091796875 - Accuracy: 0.5
